In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
from backtesting.test import GOOG
import pandas as pd
from backtesting.test import SMA
import ta


/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position



In [3]:
FB = pd.read_csv("/Users/fahad/project_repo/data/final/Moving_Average_Preparation/FB_MA_prep.csv", index_col=0)

In [4]:
FB.columns = ['Open','High','Low','Close','Volume','Date']

In [5]:
FB.index = pd.to_datetime(FB.index, format ="%Y-%m-%d")

In [6]:
FB = FB[::-1]

In [7]:
FB.head()

,Open,High,Low,Close,Volume,Date
2012-05-18,42.05,45.00,38.00,38.232,573576400,2012-05-18
2012-05-21,36.53,36.66,33.00,34.030,168192700,2012-05-21
2012-05-22,32.61,33.59,30.94,31.000,101786600,2012-05-22
2012-05-23,31.37,32.50,31.36,32.000,73600000,2012-05-23
2012-05-24,32.95,33.21,31.77,33.030,50237200,2012-05-24


In [8]:
FB['RSI']= ta.momentum.rsi(FB['Close'], n=14, fillna=True)
FB['MACD'] = ta.trend.macd_diff(FB['Close'],12,26,9,"True")

In [9]:
FB.MACD

2012-05-18    0.000000
2012-05-21   -0.041900
2012-05-22   -0.095436
2012-05-23   -0.071796
2012-05-24   -0.023300
2012-05-25   -0.032384
2012-05-29   -0.135489
2012-05-30   -0.197936
2012-05-31   -0.159354
2012-06-01   -0.198335
2012-06-04   -0.236947
2012-06-05   -0.283548
2012-06-06   -0.245661
2012-06-07   -0.225100
2012-06-08   -0.155627
2012-06-11   -0.102282
2012-06-12   -0.037438
2012-06-13    0.004906
2012-06-14    0.089859
2012-06-15    0.233610
2012-06-18    0.389376
2012-06-19    0.496514
2012-06-20    0.523104
2012-06-21    0.527316
2012-06-22    0.570311
2012-06-25    0.511102
2012-06-26    0.504823
2012-06-27    0.422060
2012-06-28    0.294650
2012-06-29    0.180228
                ...   
2019-07-05    1.097140
2019-07-08    0.910674
2019-07-09    0.937337
2019-07-10    1.098517
2019-07-11    1.012692
2019-07-12    1.102264
2019-07-15    1.001001
2019-07-16    0.839952
2019-07-17    0.522237
2019-07-18    0.186885
2019-07-19   -0.231711
2019-07-22   -0.272856
2019-07-23 

In [10]:
#dictionary to store equity
equity_dict = dict()

In [11]:
def RSI(values,n):
    """Takes values and returns an array of values with a corresponding RSI score calculated"""
    RSI = ta.momentum.rsi(values, n, fillna=True)
    
    return RSI

class SmaCrossTrailing(TrailingStrategy):
            

    def init(self):
        super().init()
        self.set_trailing_sl(4)


        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
#         self.RSI = self.I(RSI, pd.Series(self.data.Close),14)
        position = Strategy.position.__get__(self)
    

        
        if self.data.RSI > 80 and (position.is_short !=True):
                #print (self.RSI[-1])
            self.sell()
        elif self.data.RSI < 20 and (position.is_long !=True):
                #print (self.RSI[-1])
            self.buy()


            
#     def next(self):
#         #super().next()
#                 # If sma1 crosses above sma2, buy the asset
#         if crossover(self.sma1, self.sma2):
#             #print (Strategy.equity.__get__(self))
#             #print (self.data.index[-1])
#             equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
#             self.buy()

#         # Else, if sma1 crosses below sma2, sell it
#         elif crossover(self.sma2, self.sma1):
#             #print (Strategy.equity.__get__(self))
#             #print (self.data.index[-1])
#             equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
#             self.sell()
            

        
        

In [12]:
bt = Backtest(FB, SmaCrossTrailing, cash=10000, commission=.02)

bt.run()

Start                     2012-05-18 00:00:00
End                       2019-08-15 00:00:00
Duration                   2645 days 00:00:00
Exposure [%]                          19.4329
Equity Final [$]                            0
Equity Peak [$]                         10000
Return [%]                               -100
Buy & Hold Return [%]                 376.643
Max. Drawdown [%]                        -100
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    4
Win Rate [%]                               50
Best Trade [%]                         6.1644
Worst Trade [%]                      -108.614
Avg. Trade [%]                       -29.6709
Max. Trade Duration         192 days 00:00:00
Avg. Trade Duration         129 days 00:00:00
Expectancy [%]                        34.3171
SQN                                  -1.12636
Sharpe Ratio                      

In [13]:
# bt.plot(superimpose=False)

In [14]:
df = pd.DataFrame.from_dict([equity_dict])

In [15]:
df = df.T

In [16]:
df.columns = ['Equity']

In [17]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/RSI/FB_RSI_Equity.csv")

In [18]:
df

,Equity
2012-05-21,10000.000000
2012-05-22,9319.888642
2012-05-23,9620.530211
2012-05-24,9930.191028
2012-05-25,9593.472470
2012-05-29,8670.502853
2012-05-30,8475.085833
2012-05-31,8898.990446
2012-06-01,8333.784296
2012-06-04,8087.258209


In [19]:
class SmaCross(Strategy):
    
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
#         print (self.sma1)
#         print (self.sma2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            #print (Strategy.equity.__get__(self))
            #print (self.data.index[-1])
            equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            #print (Strategy.equity.__get__(self))
            #print (self.data.index[-1])
            equity_dict[self.data.index[-1]] = Strategy.equity.__get__(self)
            self.sell()